# 🧠 Deep Learning Model — CNN + BiLSTM + Multi-Head Attention
- Architecture: Conv1D → Conv1D → BiLSTM → Attention → Linear(256→168)
- Huber Loss + AdamW + ReduceLROnPlateau + Early Stopping
- Gradient clipping, dropout, Gaussian noise
- Evaluate on test, horizon-wise error, rolling origin backtesting
- Full model comparison with baselines


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os, json, time, numpy as np, joblib
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings; warnings.filterwarnings('ignore')

BASE_DIR  = '/content/drive/MyDrive/Electricity_Load_Forecast'
MODEL_DIR = os.path.join(BASE_DIR, 'models')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

In [ ]:
config         = joblib.load(os.path.join(MODEL_DIR, 'config.pkl'))
target_scaler  = joblib.load(os.path.join(MODEL_DIR, 'target_scaler.pkl'))

X_train_w = np.load(os.path.join(MODEL_DIR, 'X_train_w.npy'))
y_train_w = np.load(os.path.join(MODEL_DIR, 'y_train_w.npy'))
X_val_w   = np.load(os.path.join(MODEL_DIR, 'X_val_w.npy'))
y_val_w   = np.load(os.path.join(MODEL_DIR, 'y_val_w.npy'))
X_test_w  = np.load(os.path.join(MODEL_DIR, 'X_test_w.npy'))
y_test_w  = np.load(os.path.join(MODEL_DIR, 'y_test_w.npy'))

N_FEATURES = config['N_FEATURES']
INPUT_LEN  = config['INPUT_LEN']
OUTPUT_LEN = config['OUTPUT_LEN']
print(f"Train: {X_train_w.shape}, Val: {X_val_w.shape}, Test: {X_test_w.shape}")

In [ ]:
HP = {
    'batch_size':    64,
    'lr':            0.001,
    'weight_decay':  1e-4,
    'max_epochs':    100,
    'patience':      15,       # early stopping
    'lr_patience':   5,        # ReduceLROnPlateau
    'lr_factor':     0.5,
    'grad_clip':     1.0,
    'dropout':       0.3,
    'noise_std':     0.01,
    'conv_filters':  64,
    'lstm_hidden':   128,
    'n_heads':       4,
    'huber_delta':   1.0,
}
print("Hyperparameters:", json.dumps(HP, indent=2))

In [ ]:
def make_loader(X, y, batch_size, shuffle=True):
    ds = TensorDataset(torch.tensor(X), torch.tensor(y))
    return DataLoader(ds, batch_size=batch_size, shuffle=shuffle, num_workers=0, pin_memory=True)

train_loader = make_loader(X_train_w, y_train_w, HP['batch_size'], shuffle=True)
val_loader   = make_loader(X_val_w,   y_val_w,   HP['batch_size'], shuffle=False)
test_loader  = make_loader(X_test_w,  y_test_w,  HP['batch_size'], shuffle=False)

In [ ]:
#  MODEL ARCHITECTURE
# ═══════════════════════════════════════
class LoadForecastModel(nn.Module):
    def __init__(self, n_features, seq_len, pred_len, conv_filters=64,
                 lstm_hidden=128, n_heads=4, dropout=0.3, noise_std=0.01):
        super().__init__()
        self.noise_std = noise_std

        # Conv1D block 1: kernel=3
        self.conv1 = nn.Conv1d(n_features, conv_filters, kernel_size=3, padding=1)
        self.bn1   = nn.BatchNorm1d(conv_filters)

        # Conv1D block 2: kernel=5
        self.conv2 = nn.Conv1d(conv_filters, conv_filters, kernel_size=5, padding=2)
        self.bn2   = nn.BatchNorm1d(conv_filters)

        # BiLSTM
        self.bilstm = nn.LSTM(conv_filters, lstm_hidden, batch_first=True,
                              bidirectional=True, num_layers=1)
        lstm_out = lstm_hidden * 2  # 256

        # Multi-Head Self-Attention
        self.attention = nn.MultiheadAttention(embed_dim=lstm_out, num_heads=n_heads,
                                               batch_first=True, dropout=dropout)

        # LayerNorm + Dropout
        self.layer_norm = nn.LayerNorm(lstm_out)
        self.dropout    = nn.Dropout(dropout)

        # Output: pool → Linear(256 → 168)
        self.fc = nn.Linear(lstm_out, pred_len)

    def forward(self, x):
        # x: (B, seq_len, n_features)

        # Gaussian Noise (training only)
        if self.training and self.noise_std > 0:
            x = x + torch.randn_like(x) * self.noise_std

        # Conv1D expects (B, C, L)
        x = x.permute(0, 2, 1)                    # (B, n_features, seq_len)
        x = F.relu(self.bn1(self.conv1(x)))        # (B, 64, seq_len)
        x = F.relu(self.bn2(self.conv2(x)))        # (B, 64, seq_len)
        x = x.permute(0, 2, 1)                    # (B, seq_len, 64)

        # BiLSTM
        x, _ = self.bilstm(x)                     # (B, seq_len, 256)

        # Multi-Head Self-Attention + residual
        attn_out, _ = self.attention(x, x, x)      # (B, seq_len, 256)
        x = self.layer_norm(x + attn_out)           # residual + norm
        x = self.dropout(x)

        # Global Average Pooling
        x = x.mean(dim=1)                          # (B, 256)

        # Direct multi-step forecast
        x = self.fc(x)                             # (B, 168)
        return x

model = LoadForecastModel(
    n_features=N_FEATURES, seq_len=INPUT_LEN, pred_len=OUTPUT_LEN,
    conv_filters=HP['conv_filters'], lstm_hidden=HP['lstm_hidden'],
    n_heads=HP['n_heads'], dropout=HP['dropout'], noise_std=HP['noise_std']
).to(device)

total_params = sum(p.numel() for p in model.parameters())
print(f"Model parameters: {total_params:,}")
print(model)

In [ ]:
#  TRAINING LOOP
# ═══════════════════════════════════════
criterion = nn.HuberLoss(delta=HP['huber_delta'])
optimizer = torch.optim.AdamW(model.parameters(), lr=HP['lr'], weight_decay=HP['weight_decay'])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', patience=HP['lr_patience'], factor=HP['lr_factor'], verbose=True
)

history = {'train_loss': [], 'val_loss': [], 'lr': []}
best_val_loss = float('inf')
patience_counter = 0
best_model_path = os.path.join(MODEL_DIR, 'best_dl_model.pt')

print(f"\n{'='*60}")
print(f"Training for up to {HP['max_epochs']} epochs (early stop patience={HP['patience']})")
print(f"{'='*60}")

for epoch in range(HP['max_epochs']):
    t0 = time.time()

    # — Train —
    model.train()
    train_losses = []
    for X_b, y_b in train_loader:
        X_b, y_b = X_b.to(device), y_b.to(device)
        optimizer.zero_grad()
        pred = model(X_b)
        loss = criterion(pred, y_b)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=HP['grad_clip'])
        optimizer.step()
        train_losses.append(loss.item())
    train_loss = np.mean(train_losses)

    # — Validate —
    model.eval()
    val_losses = []
    with torch.no_grad():
        for X_b, y_b in val_loader:
            X_b, y_b = X_b.to(device), y_b.to(device)
            pred = model(X_b)
            loss = criterion(pred, y_b)
            val_losses.append(loss.item())
    val_loss = np.mean(val_losses)

    # — Scheduler —
    scheduler.step(val_loss)
    current_lr = optimizer.param_groups[0]['lr']

    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    history['lr'].append(current_lr)

    elapsed = time.time() - t0
    print(f"Epoch {epoch+1:3d}/{HP['max_epochs']} | "
          f"Train: {train_loss:.6f} | Val: {val_loss:.6f} | "
          f"LR: {current_lr:.2e} | {elapsed:.1f}s")

    # — Early Stopping —
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(model.state_dict(), best_model_path)
    else:
        patience_counter += 1
        if patience_counter >= HP['patience']:
            print(f"\n⛔ Early stopping at epoch {epoch+1} (best val loss: {best_val_loss:.6f})")
            break

# Load best model
model.load_state_dict(torch.load(best_model_path))
print(f"✅ Best model loaded (val loss: {best_val_loss:.6f})")

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
ax1.plot(history['train_loss'], label='Train Loss')
ax1.plot(history['val_loss'], label='Val Loss')
ax1.set_xlabel('Epoch'); ax1.set_ylabel('Huber Loss'); ax1.set_title('Training & Validation Loss')
ax1.legend(); ax1.grid(True, alpha=0.3)

ax2.plot(history['lr'], color='green')
ax2.set_xlabel('Epoch'); ax2.set_ylabel('Learning Rate'); ax2.set_title('Learning Rate Schedule')
ax2.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(os.path.join(MODEL_DIR, 'dl_training_history.png'), dpi=150)
plt.show()

In [ ]:
#  TEST EVALUATION
# ═══════════════════════════════════════
def predict_all(model, loader):
    model.eval()
    preds = []
    with torch.no_grad():
        for X_b, _ in loader:
            preds.append(model(X_b.to(device)).cpu().numpy())
    return np.concatenate(preds, axis=0)

def inverse_scale(y):
    return target_scaler.inverse_transform(y.reshape(-1, 1)).reshape(y.shape)

def compute_metrics(y_true, y_pred):
    yt, yp = inverse_scale(y_true), inverse_scale(y_pred)
    mae  = mean_absolute_error(yt.flatten(), yp.flatten())
    rmse = np.sqrt(mean_squared_error(yt.flatten(), yp.flatten()))
    mask = yt.flatten() != 0
    mape = np.mean(np.abs((yt.flatten()[mask] - yp.flatten()[mask]) / yt.flatten()[mask])) * 100
    peak_errors = [np.abs(yt[i, np.argmax(yt[i])] - yp[i, np.argmax(yt[i])]) for i in range(len(yt))]
    peak_mae = np.mean(peak_errors)
    return {'MAE': round(mae, 2), 'RMSE': round(rmse, 2),
            'MAPE': round(mape, 2), 'Peak_MAE': round(peak_mae, 2)}

y_pred_test = predict_all(model, test_loader)
y_pred_val  = predict_all(model, val_loader)

dl_val  = compute_metrics(y_val_w,  y_pred_val)
dl_test = compute_metrics(y_test_w, y_pred_test)

print(f"\n{'='*40}\nDL Model — Validation\n{'='*40}")
for k, v in dl_val.items():  print(f"  {k:10s}: {v}")
print(f"\n{'='*40}\nDL Model — Test\n{'='*40}")
for k, v in dl_test.items(): print(f"  {k:10s}: {v}")

In [ ]:
horizons = [1, 24, 72, 168]
yt_inv = inverse_scale(y_test_w)
yp_inv = inverse_scale(y_pred_test)

horizon_metrics = {}
for h in horizons:
    idx = h - 1  # 0-indexed
    mae_h  = mean_absolute_error(yt_inv[:, idx], yp_inv[:, idx])
    rmse_h = np.sqrt(mean_squared_error(yt_inv[:, idx], yp_inv[:, idx]))
    mask = yt_inv[:, idx] != 0
    mape_h = np.mean(np.abs((yt_inv[:, idx][mask] - yp_inv[:, idx][mask]) / yt_inv[:, idx][mask])) * 100
    horizon_metrics[h] = {'MAE': round(mae_h, 2), 'RMSE': round(rmse_h, 2), 'MAPE': round(mape_h, 2)}

print(f"\n{'='*50}\nHorizon-Wise Error (Test Set)\n{'='*50}")
print(f"{'Hour':>6} {'MAE':>10} {'RMSE':>10} {'MAPE%':>10}")
for h, m in horizon_metrics.items():
    print(f"{h:>6} {m['MAE']:>10} {m['RMSE']:>10} {m['MAPE']:>10}")

# Horizon error curve (all hours)
mae_per_hour = [mean_absolute_error(yt_inv[:, h], yp_inv[:, h]) for h in range(OUTPUT_LEN)]
plt.figure(figsize=(12, 5))
plt.plot(range(1, OUTPUT_LEN+1), mae_per_hour, 'b-', lw=1.5)
for h in horizons:
    plt.axvline(h, color='red', ls='--', alpha=0.5)
    plt.annotate(f'h={h}', (h, mae_per_hour[h-1]), fontsize=9, color='red')
plt.xlabel('Forecast Horizon (hours)'); plt.ylabel('MAE (MW)')
plt.title('Horizon-Wise MAE — DL Model (Test Set)'); plt.grid(True, alpha=0.3)
plt.savefig(os.path.join(MODEL_DIR, 'horizon_wise_error.png'), dpi=150)
plt.show()

In [ ]:
n_origins = 10
step = max(1, len(X_test_w) // n_origins)
origin_indices = list(range(0, len(X_test_w), step))[:n_origins]

rolling_metrics = []
for idx in origin_indices:
    X_single = torch.tensor(X_test_w[idx:idx+1]).to(device)
    with torch.no_grad():
        pred = model(X_single).cpu().numpy()
    true = y_test_w[idx:idx+1]
    m = compute_metrics(true, pred)
    m['origin_idx'] = idx
    rolling_metrics.append(m)

print(f"\n{'='*60}\nRolling Origin Evaluation ({n_origins} origins)\n{'='*60}")
print(f"{'Origin':>8} {'MAE':>8} {'RMSE':>8} {'MAPE%':>8} {'PeakMAE':>10}")
for rm in rolling_metrics:
    print(f"{rm['origin_idx']:>8} {rm['MAE']:>8} {rm['RMSE']:>8} {rm['MAPE']:>8} {rm['Peak_MAE']:>10}")

avg_rolling = {k: round(np.mean([m[k] for m in rolling_metrics]), 2)
               for k in ['MAE','RMSE','MAPE','Peak_MAE']}
print(f"{'AVG':>8} {avg_rolling['MAE']:>8} {avg_rolling['RMSE']:>8} "
      f"{avg_rolling['MAPE']:>8} {avg_rolling['Peak_MAE']:>10}")

In [ ]:
#  FULL MODEL COMPARISON
# ═══════════════════════════════════════
# Load baseline metrics
with open(os.path.join(MODEL_DIR, 'baseline_metrics.json'), 'r') as f:
    baseline_metrics = json.load(f)

all_metrics = {
    'Persistence': baseline_metrics['Persistence'],
    'XGBoost':     baseline_metrics['XGBoost'],
    'CNN-BiLSTM-Attn': {'val': dl_val, 'test': dl_test}
}

print(f"\n{'='*70}")
print(f"   FULL MODEL COMPARISON")
print(f"{'='*70}")
for split_name in ['val', 'test']:
    print(f"\n--- {split_name.upper()} SET ---")
    print(f"{'Model':<20} {'MAE':>8} {'RMSE':>8} {'MAPE%':>8} {'PeakMAE':>10}")
    print("-"*60)
    for model_name, m in all_metrics.items():
        t = m[split_name]
        print(f"{model_name:<20} {t['MAE']:>8} {t['RMSE']:>8} {t['MAPE']:>8} {t['Peak_MAE']:>10}")

# Comparison bar chart
fig, axes = plt.subplots(1, 4, figsize=(20, 5))
metric_names = ['MAE', 'RMSE', 'MAPE', 'Peak_MAE']
model_names = list(all_metrics.keys())
colors = ['#e74c3c', '#3498db', '#2ecc71']

for ax, metric in zip(axes, metric_names):
    vals = [all_metrics[m]['test'][metric] for m in model_names]
    bars = ax.bar(model_names, vals, color=colors, edgecolor='white', linewidth=1.5)
    ax.set_title(metric, fontsize=13, fontweight='bold')
    ax.set_ylabel(metric)
    for bar, v in zip(bars, vals):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                f'{v}', ha='center', fontsize=10)
    ax.grid(True, alpha=0.2, axis='y')

plt.suptitle('Model Comparison — Test Set', fontsize=15, fontweight='bold', y=1.03)
plt.tight_layout()
plt.savefig(os.path.join(MODEL_DIR, 'model_comparison.png'), dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# DL metrics
dl_metrics_full = {
    'val': dl_val, 'test': dl_test,
    'horizon_wise': {str(k): v for k, v in horizon_metrics.items()},
    'rolling_origin': {'per_origin': rolling_metrics, 'average': avg_rolling},
    'hyperparameters': HP,
    'total_params': total_params,
    'best_val_loss': best_val_loss
}
with open(os.path.join(MODEL_DIR, 'dl_metrics.json'), 'w') as f:
    json.dump(dl_metrics_full, f, indent=2)

# Training history
import pandas as pd
pd.DataFrame(history).to_csv(os.path.join(MODEL_DIR, 'dl_training_history.csv'), index=False)

# Full comparison
with open(os.path.join(MODEL_DIR, 'all_model_comparison.json'), 'w') as f:
    json.dump(all_metrics, f, indent=2)

print("\n✅ All saved to models/:")
print("  • best_dl_model.pt")
print("  • dl_metrics.json")
print("  • dl_training_history.csv")
print("  • dl_training_history.png")
print("  • horizon_wise_error.png")
print("  • model_comparison.png")
print("  • all_model_comparison.json")

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
for idx, ax in enumerate(axes.flat):
    si = idx * (len(y_test_w) // 4)
    ax.plot(inverse_scale(y_test_w[si]), 'k-', lw=2, label='Actual')
    ax.plot(inverse_scale(y_pred_test[si]), 'g--', lw=1.5, label='DL Prediction')
    ax.set_title(f'Test Window #{si}'); ax.set_xlabel('Hour'); ax.set_ylabel('Load (MW)')
    ax.legend(); ax.grid(True, alpha=0.3)
plt.suptitle('CNN-BiLSTM-Attention Predictions vs Actual', fontsize=14, y=1.02)
plt.tight_layout()
plt.savefig(os.path.join(MODEL_DIR, 'dl_sample_predictions.png'), dpi=150, bbox_inches='tight')
plt.show()
print("🎉 All done!")